# Using PLIO to analyze control networks
PLIO is a general purpose library for reading data from various sources. In this workshop, we will be using PLIO's ability to read ISIS control networks into a Pandas dataframe.

In [ ]:
# This allows us to interact with our plots. This must be set before importing pyplot
%matplotlib notebook

# PLIO uses pysis for some other things. We don't technically need this but it avoids a warning.
import os
os.environ['ISISROOT'] = '/usgs/cpkgs/anaconda3_linux/envs/isis4.3.0'
os.environ['ISISDATA'] = '/usgs/cpkgs/isis3/isis_data'

# This function is what reads a control network file
from plio.io.io_controlnetwork import from_isis

# General plotting library
import matplotlib
import matplotlib.pyplot as plt

# 3D plotting toolkit for matplotlib
from mpl_toolkits.mplot3d import Axes3D

# Numerical Python library
import numpy as np

# Statistics library
from scipy import stats

# Projection library for switching between rectangular and latitudinal
import pyproj

# Our networks
All of this data was generously provided by Lynn Weller and Mike Bland from their Europa control project.

The first network is a very rough starting point. The Galileo images of Europa were put through the [findfeatures](https://isis.astrogeology.usgs.gov/Application/presentation/Tabbed/findfeatures/findfeatures.html) application and then all of the resulting networks were merged together. This network has many known issues including islands, massive residuals, and poor coverage.

The second network is the final network containing Galileo and Voyager images of Europa. The issues from the initial network have been resolved and the final point cloud covers the majority of the body.

In [ ]:
galileo_net = '/scratch/jmapel/europa/networks/GLL_FFCombined_thin_SubReg2_Del_2.net'
final_net = '/scratch/jmapel/europa/networks/GalileoVoyager_Europa_Merged_2020_CilixFree.net'

# The control network dataframe

PLIO directly ingests the data from the control network file. Each row in the dataframe is a single control measure and each column is a field from the protobuf control network. The data for control points is stored implicitly in its measures.

In [ ]:
galileo_df = from_isis(galileo_net)
galileo_df.describe()

### Exercise: How many measures are there in the network? How many points are there in the network? How many images are there in the network?

tip: use len(dataframe) to find the number of rows in a dataframe

tip: use dataframe["columnName"].nunique() to find the number of unique values in a column

The protobuf control measures and control points use some of the same field names, so these fields are prefixed with point\* or measure\* (E.G. pointType and measureType).

In [ ]:
list(galileo_df.columns)

## Data types
The different columns of our dataframe store different types of data. You can look back at the original protobuf descriptions or just look directly at the data types in the dataframe.

In [ ]:
galileo_df.dtypes

Most of the column data types are intuitive but there are several that need explanation

**pointType, measureType, aprioriSurfPointSource, and aprioriRadiusSource** are int64 data, but actually these are enumerations. For example, a pointType of 2 means Free. For now, you has to use the actual protobuf code to convert them but we are working to add this directly to PLIO.

In [ ]:
galileo_df[['pointType', 'measureType', 'aprioriSurfPointSource']].head()

In [ ]:
from plio.io.ControlPointFileEntryV0005_pb2 import ControlPointFileEntryV0005 as protoPoint
print(f'A pointType value of 2 is named {protoPoint.PointType.Name(2)}')
print(f'A measureType named RegisteredSubPixel has a value of {protoPoint.Measure.MeasureType.Value("RegisteredSubPixel")}')
print(f'An aprioriSurfPointSource value of 0 is named {protoPoint.AprioriSource.Name(0)}')
print('')
print('pointType names and values')
print('==========================')
for value in protoPoint.PointType.values():
    print(f'{value} | {protoPoint.PointType.Name(value)}')
print('')

print('measureType names and values')
print('============================')
for value in protoPoint.Measure.MeasureType.values():
    print(f'{value} | {protoPoint.Measure.MeasureType.Name(value)}')
print('')
    
print('aprioriSurfPointSource/aprioriRadiusSource names and values')
print('===========================================================')
for value in protoPoint.AprioriSource.values():
    print(f'{value} | {protoPoint.AprioriSource.Name(value)}')

### Exercise: Have any measure in this network been sub-pixel registered?

tip: look at the measure types

**id, pointChoosername, pointDatetime, aprioriSurfPointSourceFile, aprioriRadiusSourceFile, serialnumber, measureChoosername, and measureDatetime** are all listed as objects but are simply strings.

In [ ]:
galileo_df[['id', 'serialnumber', 'pointChoosername', 'pointDatetime', 'measureChoosername', 'measureDatetime']].head()

**adjustedCovar, pointLog, and measureLog** are more complicated. We will go over adjustedCovar later with the final Euroap network. pointLog is leftover from older network formats and can be ignored. measureLog contains information about the registration of the measure.

In [ ]:
galileo_df.loc[1,'measureLog']

## Data availability
Depending on how your network was generated and what processing has been done, many fields will not be set. If a numerical field has a value of 0, then it has not been set. For example, in our network has not been bundle adjusted, so there are only apriori ground points.

In [ ]:
galileo_df[['aprioriX', 'aprioriY', 'aprioriZ', 'adjustedX', 'adjustedY', 'adjustedZ']].describe()

### Exercise: Can you find all of the fields that are completely unset in our control network?

tip: numerical fields default to 0, strings default to an empty string "", boolean values default to False, and enums default to their first value.

# Looking at bundle adjust results

Our Galileo network is interesting but networks have significantly more useful information in them after bundle adjustment. So, let's take a look at the final Europa network.

In [ ]:
final_net_df = from_isis(final_net)
final_net_df.describe()

### Exercise: What fields are set in the bundle adjusted network that weren't previously?

## Analyzing the measures
The data in a control network dataframe is not always in the format we want to work with. The measure residuals are broken down into the line and sample residuals. The following cells compute the full magnitude of the residuals and adds it to the dataframe. Then, they plot the residuals and color points by the magnitude of the residuals.

In [ ]:
final_net_df['residualMag'] = np.sqrt(final_net_df['sampleResidual']**2 + final_net_df['lineResidual']**2)

In [ ]:
resid_fig = plt.figure(figsize=(6, 6))
resid_ax = resid_fig.add_subplot(111)
resid_scatter = resid_ax.scatter(final_net_df['sampleResidual'], final_net_df['lineResidual'], c=final_net_df['residualMag'], marker='+')
resid_ax.set_aspect('equal')
plt.axhline(0, color='black')
plt.axvline(0, color='black')
resid_cbar = plt.colorbar(resid_scatter)
resid_fig.suptitle('Bundle Adjusted Measure Residuals')
resid_ax.set_xlabel('Sample Residual')
resid_ax.set_ylabel('Line Residual')
resid_cbar.set_label('Residual Magnitude')
plt.show()

We can also color our points based on other properties. The following cell determines which measures belong to which mission. Then, the next cell plots the residuals again but colors them based on if they are from Galileo, Voyager1, or Voyager2.

In [ ]:
is_galileo = final_net_df.apply(lambda row: row['serialnumber'][:7]  == 'Galileo', axis=1) # Boolean array indicating if each measure is a Galileo image
is_voyager1 = final_net_df.apply(lambda row: row['serialnumber'][:8] == 'Voyager1', axis=1) # Boolean array indicating if each measure is a Voyager1 image
is_voyager2 = final_net_df.apply(lambda row: row['serialnumber'][:8] == 'Voyager2', axis=1) # Boolean array indicating if each measure is a Voyager2 image

In [ ]:
inst_resid_fig = plt.figure(figsize=(6, 6))
inst_resid_ax = inst_resid_fig.add_subplot(111)
inst_resid_ax.scatter(final_net_df.loc[is_galileo]['sampleResidual'], final_net_df.loc[is_galileo]['lineResidual'], color='Red', marker='+', alpha=0.25, label='Galileo')
inst_resid_ax.scatter(final_net_df.loc[is_voyager1]['sampleResidual'], final_net_df.loc[is_voyager1]['lineResidual'], color='Blue', marker='+', alpha=0.25, label='Voyager1')
inst_resid_ax.scatter(final_net_df.loc[is_voyager2]['sampleResidual'], final_net_df.loc[is_voyager2]['lineResidual'], color='Green', marker='+', alpha=0.25, label='Voyager2')
inst_resid_ax.set_aspect('equal')
plt.axhline(0, color='black')
plt.axvline(0, color='black')
plt.legend()
inst_resid_fig.suptitle('Bundle Adjusted Measure Residuals by Mission')
inst_resid_ax.set_xlabel('Sample Residual')
inst_resid_ax.set_ylabel('Line Residual')
plt.show()

### Exercise: What is the mean residual magnitude of the Galileo measures? What about for Voyager 1 and Voyager 2?

tip: you can use the is_galileo, is_voyager1, and is_voyager2 arrays to index the network dataframe

We can even test if the measure residuals are normally distributed

In [ ]:
alpha = 1e-3 # 99.999% confidence
_, normal_test_result = stats.normaltest(final_net_df['residualMag'])
print(f'Chi-squared test statistic: {normal_test_result}')
if (normal_test_result < alpha):
    print("The residuals are normally distributed")
else:
    print("The residuals may not be normally distributed")

## Analyzing the points
The information for control points is duplicated for each measure they have. So, the first step in looking at control point data is to extract only the data we want from the dataframe. This will make the dataframe easier to read and it will make things run quicker.

In [ ]:
point_columns = ['id',
                 'pointType',
                 'pointChoosername',
                 'pointDatetime',
                 'pointEditLock',
                 'pointIgnore',
                 'pointJigsawRejected',
                 'aprioriSurfPointSource',
                 'aprioriSurfPointSourceFile',
                 'aprioriRadiusSource',
                 'aprioriRadiusSourceFile',
                 'latitudeConstrained',
                 'longitudeConstrained',
                 'radiusConstrained',
                 'aprioriX',
                 'aprioriY',
                 'aprioriZ',
                 'aprioriCovar',
                 'adjustedX',
                 'adjustedY',
                 'adjustedZ',
                 'adjustedCovar',
                 'pointLog']
final_points_df = final_net_df[point_columns].drop_duplicates('id')

Next, we're going to transform the point data so that it's more useful to us. This cell will take the (X, Y, Z) adjusted ground points and convert them to (lat, lon, radius) using a library called pyproj. pyproj is a very powerful projections library and can do many cartofraphic transformations and projections.

**Note: This cell will generate a warning because we are using old pyproj.Proj calls which will eventually need to change. For now we can ignore the warning.**

In [ ]:
# Compute the lat/lon/alt
europa_radii = [1562600, 1560300, 1559500]
ecef = pyproj.Proj(proj='geocent', a=europa_radii[0], b=europa_radii[1], c=europa_radii[2])
lla = pyproj.Proj(proj='latlong', a=europa_radii[0], b=europa_radii[1], c=europa_radii[2])
lon, lat, alt = pyproj.transform(ecef, lla, final_points_df['adjustedX'].values, final_points_df['adjustedY'].values, final_points_df['adjustedZ'].values, radians=True)

# Store the data in the dataframe
final_points_df['latitude'] = lat
final_points_df['longitude'] = lon
final_points_df['altitude'] = alt

# We will also want the point radii
final_points_df['radius'] = np.sqrt(final_points_df['adjustedX']**2 + final_points_df['adjustedY']**2 + final_points_df['adjustedZ']**2)

Similar to how we computed the residual magnitude, we want to compute the average residual magnitude for each point. The following cell goes back to our original dataframe, computes the mean point by point, and then saves all of the results in our new dataframe.

**Note: This cell can take a while to run because it has to re-access the dataframe for every point**

In [ ]:
final_points_df["averageResidual"] = 0
for point_id, group in final_net_df.groupby('id'):
    final_points_df.loc[final_points_df.id == point_id, "averageResidual"] = group['residualMag'].mean()

### Exercise: What is the 95% percentile for the point average residuals?

## Plotting the points
Now that we have latitudes and longitudes for each point, we can generate some simple plots to look at them.

In [ ]:
point_map = plt.figure(figsize=(10, 10))
point_ax = point_map.add_subplot(111)
point_ax.scatter(np.rad2deg(lon), np.rad2deg(lat), marker='+')
point_map.suptitle('Control Points')
point_ax.set_xlabel('Longitude')
point_ax.set_ylabel('Latitude')
plt.show()

It can also be helpful to color the points based on different values. The following cell draws the same plot but colors each point based on its average residual. Because the residuals are not uniformly distributed we also apply a lograithmic scale to the colors that you can see in the colorbar.

In [ ]:
point_resid_map = plt.figure(figsize=(10, 10))
point_resid_ax = point_resid_map.add_subplot(111)
point_resid_norm = matplotlib.colors.LogNorm(vmax=final_points_df["averageResidual"].max())
point_resid_scatter = point_resid_ax.scatter(np.rad2deg(lon), np.rad2deg(lat), c=final_points_df["averageResidual"], alpha=0.5, norm=point_resid_norm, marker='+', cmap=plt.get_cmap('plasma'))
point_resid_cbar = plt.colorbar(point_resid_scatter)
point_resid_map.suptitle('Control Points')
point_resid_ax.set_xlabel('Longitude')
point_resid_ax.set_ylabel('Latitude')
point_resid_cbar.set_label('Average Residual Magnitude')
plt.show()

## 3D Plotting
2D plotting either requires these simple equal area projections or converting to another projection via pyproj. Instead, let's look at our data in true 3D.

The following cell plots the control points in 3D and colors them based on their altitude, in meters.

In [ ]:
fig = plt.figure(figsize=(10, 10))
ax = fig.add_subplot(111, projection='3d')
alt_norm = matplotlib.colors.TwoSlopeNorm(vmin=np.min(alt), vcenter=0, vmax=np.max(alt))
p = ax.scatter(final_points_df['adjustedX'], final_points_df['adjustedY'], final_points_df['adjustedZ'], c=alt, norm=alt_norm, cmap=plt.get_cmap('Spectral'), alpha=0.1)
alt_cbar = plt.colorbar(p)
fig.suptitle('3D Control Points')
alt_cbar.set_label('Altitude (m)')
plt.show()

## Fitting new radii
Now that we have looked at all of the point data and it seems to be in tip-top shape, let's solve for updated radii. The scipy library has a very easy to use least squares function that we can use.

Ideally we would weight all of these based on the radii sigma from our control network using the much more complex [scipy.optimize.least_squares](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.least_squares.html) function because it allows for output uncertainties, but that is far beyond this example.

In [ ]:
num_points = len(final_points_df)
x2 = np.reshape(final_points_df['adjustedX'].values**2, (num_points, 1))
y2 = np.reshape(final_points_df['adjustedY'].values**2, (num_points, 1))
z2 = np.reshape(final_points_df['adjustedZ'].values**2, (num_points, 1))
A = np.hstack([x2, y2, z2])
b = np.ones(num_points)
x = np.linalg.lstsq(A, b, rcond=None)[0].squeeze()
updated_radii = np.reciprocal(np.sqrt(x))
print(f'New Europa radii {updated_radii}')
print(f'Difference between old and new radii {updated_radii - europa_radii}')